In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from keras_OneCycle import OneCycle
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
import keras
#import tensorflow

import tensorflow.keras.models as M
import tensorflow.keras.layers as L
from keras import backend as K
#keras.backend

#import tensorflow as tf
#from tensorflow import keras #<------- use this for tensorflow



import matplotlib.pyplot as plt
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns

Using plaidml.keras.backend backend.


In [2]:
cd data

/Users/kush/Desktop/DS/inpredo-master/RNN_tabular/AI-for-Trading/data


In [3]:
data = pd.read_csv("ND100.M5.csv", parse_dates=True,  names=["open", "high", "low", "close", "volume"])
data.reset_index(drop=True, inplace=True)
data = data[["close", "volume"]]

In [4]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1, predict_forward=3, column=3):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-predict_forward):
        j = i + look_back
        a = dataset[i:j,:]
        dataX.append(a)
		
        # for predicting actual price
        dataY.append(dataset[j:j + predict_forward, column])

        # for % change
        #pct = (-data[i + look_back,3] + data[i + look_back+ predict_forward, 3])/data[i + look_back,3]
        #if data[i + look_back,3] == 0:
        #	print(i,"zero")

    return np.array(dataX), np.array(dataY)

In [5]:
# fix random seed for reproducibility
np.random.seed(42)
normalize_max = data.max().max()//1000 
# normalize the dataset
x_scaler = MinMaxScaler(feature_range=(0,normalize_max))
data = x_scaler.fit_transform(data)

# remove values that are 0
data = np.delete(data,np.where(data==0)[0],axis=0)

In [6]:
n_bars = 10
predict_forward = 1
x_train, y_train = create_dataset(data,n_bars, predict_forward=predict_forward, column=0)

#remove the last sample one bc it may not have enough  y_train values (if the dataset size doenst evenly divide into the sample size)
x_train = x_train[:-predict_forward]
y_train = np.stack(y_train[:-predict_forward])

In [7]:
# scale y_train
y_scaler = MinMaxScaler(feature_range=(0,normalize_max))
y_train = y_scaler.fit_transform(y_train.reshape(-1,1))

In [8]:
x_train[100]

array([[1.18499948, 0.17214765],
       [1.18809025, 0.07852349],
       [1.19097496, 0.05838926],
       [1.1983928 , 0.0533557 ],
       [1.19942306, 0.07449664],
       [1.20148357, 0.05033557],
       [1.19488993, 0.05939597],
       [1.19159312, 0.05637584],
       [1.19571414, 0.03624161],
       [1.19736255, 0.0533557 ]])

In [21]:
# Enter the number of samples, training + validation
nb_train_samples = round(len(x_train)*.8)
nb_val_samples = round(len(x_train)*.2)

# We have 3 classes, buy sell, and hold
n_features = x_train[0].shape[1]
num_classes = 3
epochs = 200
batch_size = 64
regularizer = keras.regularizers.l2()
# define model
def make_model(input_shape):

    model = keras.models.Sequential()
    #model.add(tf.compat.v1.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='selu'))
    #model.add(keras.layers.MaxPooling1D(pool_size=2))
    model.add(keras.layers.GRU(10,input_shape=input_shape, recurrent_regularizer=regularizer, return_sequences=True, recurrent_dropout=0.9))
    model.add(keras.layers.GRU(10,input_shape=input_shape, recurrent_regularizer=regularizer, recurrent_dropout=0.9))
    model.add(keras.layers.Dense(1))
    
    return model

model = make_model(input_shape=x_train.shape[1:])
#model.build(input_shape=x_train.shape[:])

callbacks = [
    #OneCycle(min_lr=7e-6, max_lr=7e-2, min_mtm = 0.85, max_mtm = 0.95, annealing_stage=0.1, annealing_rate=0.01, training_iterations=np.ceil(((x_train.shape[0]*epochs)/(batch_size)))),

    keras.callbacks.ReduceLROnPlateau(monitor="val_mean_squared_error", factor=0.5, patience=5, min_lr=0.0001),
    keras.callbacks.EarlyStopping(monitor="mean_squared_error", patience=50, verbose=1),
]

opt = keras.optimizers.Adam()
#losses = keras.losses.MeanSquaredError()

model.compile(optimizer=opt, loss="mse", metrics=["mse"])

keras.utils.plot_model(model, show_shapes=True)
model.summary()

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
#model.save("my_model")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("my_model")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_11 (GRU)                 (None, 10, 10)            390       
_________________________________________________________________
gru_12 (GRU)                 (None, 10)                630       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 1,031
Trainable params: 1,031
Non-trainable params: 0
_________________________________________________________________


In [22]:
# fit model : mse - 7.5e-4 : valmse - .07 : (2x 10 neurons with dropout0.9)
#model = keras.models.load_model('model')
history = model.fit(x_train, y_train, epochs=500, verbose=1, batch_size=128, callbacks=callbacks, validation_split=0.2)
model.save('modelsave')

Train on 80214 samples, validate on 20054 samples
Epoch 1/500
80214/80214 [==============================] - 22s 272us/step - loss: 2.9891 - mean_squared_error: 2.8613 - val_loss: 13.0159 - val_mean_squared_error: 12.9404
Epoch 2/500
80214/80214 [==============================] - 13s 161us/step - loss: 0.3253 - mean_squared_error: 0.2709 - val_loss: 7.3139 - val_mean_squared_error: 7.2748
Epoch 3/500
80214/80214 [==============================] - 13s 167us/step - loss: 0.1542 - mean_squared_error: 0.1246 - val_loss: 5.1828 - val_mean_squared_error: 5.1611
Epoch 4/500
80214/80214 [==============================] - 13s 167us/step - loss: 0.0873 - mean_squared_error: 0.0710 - val_loss: 3.8829 - val_mean_squared_error: 3.8711
Epoch 5/500
80214/80214 [==============================] - 14s 170us/step - loss: 0.0537 - mean_squared_error: 0.0450 - val_loss: 2.8575 - val_mean_squared_error: 2.8512
Epoch 6/500
80214/80214 [==============================] - 14s 169us/step - loss: 0.0334 - mean_sq

KeyboardInterrupt: 

In [ ]:
# show traning/validation plots
metric = "mean_squared_error"
plt.figure(figsize=(10,10))
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

In [ ]:
y_train[-i].shape

In [ ]:
# get latest prediction
i = 80
x_pred = x_train[-i]
prediction = model.predict(x_pred.reshape(1,10,5))
y_pred = y_scaler.inverse_transform(prediction)
y_actual = y_train[-i]
print("Prediction: ",y_pred[0][0])
print( "Actual: ", y_actual[0])
print("MSE: ", (y_actual[0]- y_pred[0][0])**2)

# plot latest prediction
#plt.plot(np.arange(1,y_pred.shape[1]+1),y_pred[0]);
#plt.plot(np.arange(y_pred.shape[1]+1,len(y_actual[0])+1),y_actual[0]);

0.02554094176668921

In [ ]:
pred_df = pd.DataFrame()
pred_df["actual"] = x_train[-1][:,3]

tmp = np.empty(len(pred_df.values)-predict_forward)
tmp.fill(np.nan)
tmp = np.concatenate((tmp, y_pred[0]))

pred_df["predicted"] = tmp

In [34]:
y_actual[0],y_pred[0][0]

(0.1033183384035478, 0.26313367)

In [ ]:
tmp